# Notebook for quantitative evaluation of document and aspect based sentiment analysis using labels assigned manually to articles.

This prepares confusion matrices.

### Imports

In [ ]:
import numpy as np
import pandas as pd
import os
import ast

# Evaluation of ABSA

### Read data

You might need to specify the directory.

In [ ]:
df_test_absa_model = pd.read_csv(os.path.join('..', 'aspect_based_sentiment_analysis', 'testset_results_absa.csv'))

In [ ]:
df_test_absa_model

The chunk of code below assigns to each article a class based od model output.

In [ ]:
results_dict = {}
for i, row in df_test_absa_model.iterrows():
    keywords = ast.literal_eval(row.keywords_sentiment)[0]
    ners = ast.literal_eval(row.ner_sentiment)[0]
    #keywords = [keyword.strip() for keyword in keywords]

    aspects = keywords | ners

    label = None
    score = -1
    results = {}
    for aspect in aspects.keys():
        #print(aspect)
        #print(aspects[aspect][0])
        for l in aspects[aspect][0]:
            if l['score'] > score:
                score = l['score']
                label = l['label']
        #numeric_label = None
        if label == 'Negative':
            numeric_label = -1
        elif label == 'Neutral':
            numeric_label = 0
        else:
            numeric_label = 1
        results[aspect] = numeric_label

    results_dict[row['Unnamed: 0']] = (results)


In [ ]:
results_dict

In [ ]:
df_test_absa_model

The chunk of code below reads the data from files filled by annotators. This will be treated as ground truth and evaluated against it.

In [ ]:
df_test_annotated = pd.DataFrame()
for file in os.listdir('./NLP'):
    df_test_annotated = pd.concat([df_test_annotated, pd.read_excel('./NLP' + '/' +file)])
print(df_test_annotated)

In [ ]:
df_test_annotated

Code below is responsible for parsing the text from files prepared by labelers.

In [ ]:
results_annotation = {}
results_annotation_overall = {}
for i, row in df_test_annotated.iterrows():
    keywords = ast.literal_eval(row.keywords_lower)
    ners = ast.literal_eval(row.ner_list)

    aspects = keywords + ners

    results = {}
    for aspect in aspects:
        results[aspect.split(':')[0]] = aspect.split(':')[1] if ':' in aspect else None

    results_annotation[row['Unnamed: 0']] = (results)
    results_annotation_overall[row['Unnamed: 0']] = float(row.overall)


In [ ]:
results_annotation_overall

In [ ]:
results_annotation

In [ ]:
results_dict

In [ ]:
def eval_class_1_vs_0(annotated, pred, tp, fp, fn, tn):

    if annotated == '1' and pred == 1:
        tp += 1
    if annotated == '1' and pred == 0:
        fn += 1
    if annotated == '0' and pred == 1:
        fp += 1
    if annotated == '0' and pred == 0:
        tn += 1

    return tp, fp, fn, tn

def eval_class_1_vs_min1(annotated, pred, tp, fp, fn, tn):

    if annotated == '1' and pred == 1:
        tp += 1
    if annotated == '1' and pred == -1:
        fn += 1
    if annotated == '-1' and pred == 1:
        fp += 1
    if annotated == '-1' and pred == -1:
        tn += 1

    return tp, fp, fn, tn

def eval_class_0_vs_min1(annotated, pred, tp, fp, fn, tn):

    if annotated == '0' and pred == 0:
        tp += 1
    if annotated == '0' and pred == -1:
        fn += 1
    if annotated == '-1' and pred == 0:
        fp += 1
    if annotated == '-1' and pred == -1:
        tn += 1

    return tp, fp, fn, tn

In [ ]:
def eval_core(results_dict, results_annotation, type):
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    for i in results_dict.keys():
        for j in results_dict[i].keys():
            try:
                if results_annotation[i][j] is None:
                    continue
            except Exception as e:
                print(e)
                continue

            if type == '1vs0':
                tp, fp, fn, tn = eval_class_1_vs_0(results_annotation[i][j], results_dict[i][j], tp, fp, fn, tn)
            elif type == '1vsmin1':
                tp, fp, fn, tn = eval_class_1_vs_min1(results_annotation[i][j], results_dict[i][j], tp, fp, fn, tn)
            elif type == '0vsmin1':
                tp, fp, fn, tn = eval_class_0_vs_min1(results_annotation[i][j], results_dict[i][j], tp, fp, fn, tn)


    return tp, fp, fn, tn

In [ ]:
tp, fp, fn, tn = eval_core(results_dict, results_annotation, '1vs0')
print(tp, fn, fp, tn)

In [ ]:
tp, fp, fn, tn = eval_core(results_dict, results_annotation, '1vsmin1')
print(tp, fn, fp, tn)

In [ ]:
tp, fp, fn, tn = eval_core(results_dict, results_annotation, '0vsmin1')
print(tp, fn, fp, tn)

# Document based sentiment analysis evaluation.
This time the code is much simpler.

### Read data, you might need to specify the path.

In [ ]:
df_test_doc_model = pd.read_csv('../document_based_sentiment_analysis/test_df_overall_sentiment.csv')

In [ ]:
df_test_doc_model

Preparing results as dict.

In [ ]:
results_doc_model = {}
for i, row in df_test_doc_model.iterrows():

    results_doc_model[row['Unnamed: 0']] = row.overall_sentiment


In [ ]:
results_doc_model

In [ ]:
results_annotation_overall

The rules are a bit different. In document based analysis, the model selected so far did not have the neutral class. Therefore, we treat the neutral class assigned by labelers as positive to be able to compare it. In the future, we will try to switch to a model capable of assigning neutral class.

In [ ]:
tp = 0
fp = 0
fn = 0
tn = 0

for k in results_annotation_overall.keys():
    if not np.isnan(results_annotation_overall[k]):

        if results_doc_model[k] == 1 and results_annotation_overall[k] != -1:
            tp += 1
        if results_doc_model[k] == 1 and results_annotation_overall[k] == -1:
            fp += 1
        if results_doc_model[k] == 0 and results_annotation_overall[k] != -1:
            fn += 1
        if results_doc_model[k] == 0 and results_annotation_overall[k] == -1:
            tn += 1

In [ ]:
print(tp, fn, fp, tn)

In [ ]:
# tp = 0
# fp = 0
# fn = 0
# tn = 0
#
# for k in results_annotation_overall.keys():
#     if not np.isnan(results_annotation_overall[k]):
#         #print(results_annotation_overall[k])
#         if results_doc_model[k] == 1 and results_annotation_overall[k] == 1:
#             tp += 1
#         if results_doc_model[k] == 1 and results_annotation_overall[k] != 1:
#             fp += 1
#         if results_doc_model[k] == 0 and results_annotation_overall[k] == 1:
#             fn += 1
#         if results_doc_model[k] == 0 and results_annotation_overall[k] != 1:
#             tn += 1

In [ ]:
# print(tp, fn, fp, tn)